In [2]:
from twython import TwythonStreamer  
import pandas as pd

In [13]:
tweet_data = {}
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = data
            created_at = str(time.strftime("%I:%M:%S %p")).decode('cp1252').encode('utf-8') + ' - ' + str(time.strftime("%d %B %Y")).decode('cp1252').encode('utf-8')
            #self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    #def save_to_csv(self, tweet):
    #    with open(r'saved_tweets.csv', 'a', encoding="utf-8") as file:
    #        writer = csv.writer(file)
    #        writer.writerow(list(tweet.values()))

In [14]:
credentials = {}  
credentials['CONSUMER_KEY'] = 'eKq9jt7MZYpXiBlr7uzXB33DC' 
credentials['CONSUMER_SECRET'] = '4Qqig1M3cXMXYlMuH9QGb3LoXIQ2sfB4OHTO3TjDW4VxQxjAan' 
credentials['ACCESS_TOKEN'] = '2885435940-AeKqojbbcIZtitzFqYokhokX0CCNZfFmTlYLdbm'  
credentials['ACCESS_SECRET'] = 'hrsSyIshMlf7COG3Sln5phVUIoA0gksHUexfZBWqfb5Ye'

In [15]:
def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d

In [18]:
# Instantiate from our streaming class
stream = MyStreamer(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'],  
                    credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='python',filter_level=1)  

KeyboardInterrupt: 

In [19]:
search_phrase = '#makedonalddrumpfagain'
time_limit = 1.0 # runtime limit in hours
min_days_old, max_days_old = 1, 2 # search limits
 
# e.g. min_days_old, max_days_old = 7, 8
# gives the current weekday from last week,
# min_days_old=0 will search from right now

In [21]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys

In [22]:
def load_api():
    ''' Function that loads the twitter API after authorizing
        the user. '''
 
    consumer_key = 'eKq9jt7MZYpXiBlr7uzXB33DC' 
    consumer_secret = '4Qqig1M3cXMXYlMuH9QGb3LoXIQ2sfB4OHTO3TjDW4VxQxjAan' 
    access_token = '2885435940-AeKqojbbcIZtitzFqYokhokX0CCNZfFmTlYLdbm'  
    access_secret = 'hrsSyIshMlf7COG3Sln5phVUIoA0gksHUexfZBWqfb5Ye'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

In [29]:
load_api()

In [25]:
def tweet_search(api, query, max_tweets, max_id, since_id):
    ''' Function that takes in a search string 'query', the maximum
        number of tweets 'max_tweets', and the minimum (i.e., starting)
        tweet id. It returns a list of tweepy.models.Status objects. '''
 
    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
                                    max_id=str(max_id-1))
#                                    geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(17*60)
            break # stop the loop
    return searched_tweets, max_id

In [ ]:
tweet_search(load_api(),'python','150','12345')

In [26]:
def get_tweet_id(api, date='', days_ago=9, query='a'):
    ''' Function that gets the ID of a tweet. This ID can
        then be used as a 'starting point' from which to
        search. The query is required and has been set to
        a commonly used word by default. The variable
        'days_ago' has been initialized to the maximum amount
        we are able to search back in time (9).'''
 
    if date: # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0&amp;gt;2}-{2:0&amp;gt;2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0&amp;gt;2}-{2:0&amp;gt;2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0].id

In [27]:
def write_tweets(tweets, filename):
    ''' Function that appends tweets to a file. '''
 
    with open(filename, 'a') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')

In [51]:
import json
 
tweet_files = ['python_2019-03-01.json']
tweets = []
for file in tweet_files:
    with open(file, 'r') as f:
        for line in f.readlines():
            tweets.append(json.loads(line))
 

In [63]:
def populate_tweet_df(tweets):
    df = pd.DataFrame()

     
    df['hashtags'] = list(map(lambda tweet: ", ".join(hashtag['text'] for hashtag in tweet['entities']['hashtags']).strip(", ")
                        if tweet['entities'] != None else 'NaN', tweets))
    
    df['user'] = list(map(lambda tweet: tweet['user']['screen_name']
                        if tweet['user'] != None else 'NaN', tweets))
    
    df['text'] = list(map(lambda tweet: tweet['text'], tweets))
 
    df['location'] = list(map(lambda tweet: tweet['user']['location'], tweets))
 
    df['country_code'] = list(map(lambda tweet: tweet['place']['country_code']
                                  if tweet['place'] != None else '', tweets))
 
    df['long'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][0]
                        if tweet['coordinates'] != None else 'NaN', tweets))
 
    df['latt'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][1]
                        if tweet['coordinates'] != None else 'NaN', tweets))
    
    df['created_at']=list(map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
                        if tweet['created_at'] != None else 'NaN', tweets))
    
    
 
    return df

In [64]:
tweets_processed=populate_tweet_df(tweets)

In [65]:
tweets_processed.head()

,hashtags,user,text,location,country_code,long,latt,created_at
0,"python, javascript, angular, angularjs, reactj...",jcgo810326,RT @ProgrammerBooks: Doing Data Science [PDF] ...,Lima,,NaN,NaN,2019-03-01 23:59:57
1,,Dolkin1,@Follow_Follow_ For me life of Brian is the be...,glasgow,,NaN,NaN,2019-03-01 23:59:36
2,ipyleaflet,pascalbugnion,RT @Renou_Martin: Today's the release day :D #...,"London, England",,NaN,NaN,2019-03-01 23:59:35
3,,seidodge,This is hilarious. Like a monty python sketch ...,"Dublin, Ireland",,NaN,NaN,2019-03-01 23:58:59
4,,aglogan,Not something you see every day. Up at St Ives...,"Sydney, Australia",AU,151.187,-33.7056,2019-03-01 23:58:11


In [66]:
tweets_processed.to_csv('Tweets.csv')

In [59]:
def process_tweet(tweet): 
    date = time.strftime('%Y-%m-%d', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
    date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
    #print(date)
    if date >'2019-02-05':
        #print("yes")
        hashtags = ", ".join(hashtag['text'] for hashtag in tweet['entities']['hashtags'])
        hashtags = hashtags.strip(", ")
        d = pd.DataFrame({'created_at':[date_time], 'hashtags':[hashtags], 'text':[tweet['text']],
                          'user': [tweet['user']['screen_name']],
                          'user_loc':[tweet['user']['location']]})
    
        return d

In [61]:
process_tweet(tweets)

TypeError: list indices must be integers or slices, not str